In [8]:
# LOAD LIBRARIES
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime, timedelta


In [26]:
def fetch_data_with_prior_days(ticker_symbol, start_date, end_date, prior_days=20):
    """
    Fetch financial data for a ticker between a start and end date,
    including `prior_days` of data before the start date.

    Parameters:
    - ticker_symbol: str, the stock ticker symbol (e.g., '^FCHI').
    - start_date: str, start date for the main data range (e.g., '2000-01-01').
    - end_date: str, end date for the main data range (e.g., '2024-01-01').
    - prior_days: int, number of days before the start date to include (to avoid NA when adding technical indicators).

    Returns:
    - pd.DataFrame, combined dataset including `prior_days` before the start date.
    """
    # Convert start_date to datetime for manipulation
    start_datetime = datetime.strptime(start_date, '%Y-%m-%d')

    # Calculate the range for prior data
    prior_start_date = (start_datetime - timedelta(days=prior_days * 2)).strftime('%Y-%m-%d')
    prior_end_date = (start_datetime - timedelta(days=1)).strftime('%Y-%m-%d')

    # Fetch prior data
    prior_data = yf.download(ticker_symbol, start=prior_start_date, end=prior_end_date)

    # Fetch main data
    main_data = yf.download(ticker_symbol, start=start_date, end=end_date)

    # Take only the last `prior_days` rows of prior data
    prior_data = prior_data.tail(prior_days)

    # Concatenate the datasets
    combined_data = pd.concat([prior_data, main_data])

    # Reset index and flatten columns if necessary
    combined_data.reset_index(inplace=True)
    if isinstance(combined_data.columns, pd.MultiIndex):
        combined_data.columns = combined_data.columns.get_level_values(0)

    return combined_data


In [32]:
# Define parameters
ticker_symbol = '^FCHI'
start_date = '2000-01-01'
end_date = '2024-10-31'

# Fetch data including 20 days before the start_date
data = fetch_data_with_prior_days(ticker_symbol, start_date, end_date, prior_days=20)

# Add log-return
data['Return'] = np.log(data['Close'] / data['Close'].shift(1))

# Display the first and last rows of the dataset
print(data.head())
print(data.tail())


[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed

Price       Date        Close         High          Low         Open  Volume  \
0     1999-12-03  5468.049805  5471.640137  5360.330078  5360.330078       0   
1     1999-12-06  5509.970215  5513.379883  5445.220215  5462.490234       0   
2     1999-12-07  5534.509766  5559.169922  5499.729980  5507.729980       0   
3     1999-12-08  5487.120117  5520.779785  5472.319824  5501.490234       0   
4     1999-12-09  5548.620117  5558.049805  5461.229980  5476.000000       0   

Price    Return  
0           NaN  
1      0.007637  
2      0.004444  
3     -0.008599  
4      0.011146  
Price       Date        Close         High          Low         Open  \
6363  2024-10-24  7503.279785  7565.390137  7503.020020  7527.640137   
6364  2024-10-25  7497.540039  7518.750000  7459.410156  7489.870117   
6365  2024-10-28  7556.939941  7580.680176  7494.930176  7542.129883   
6366  2024-10-29  7511.109863  7618.319824  7506.520020  7593.859863   
6367  2024-10-30  7428.359863  7470.689941  7377.39

In [33]:
import numpy as np
from ta.momentum import RSIIndicator, StochasticOscillator

def get_technical_indicators(dataset, shift_values=True):
    """
    Function to generate technical indicators with an option to shift values.

    Parameters:
    - dataset (pd.DataFrame): Input dataset containing 'Close', 'Open', 'High', and 'Low'.
    - shift_values (bool): If True, shifts all calculated indicators by 1 period.

    Returns:
    - pd.DataFrame: Dataset with technical indicators.
    """
    # Create 5, 10 and 20 days Moving Average
    dataset['SMA_5'] = dataset['Close'].rolling(window=5).mean()
    dataset['SMA_10'] = dataset['Close'].rolling(window=10).mean()
    dataset['SMA_20'] = dataset['Close'].rolling(window=20).mean()

    # Create 5, 10 and 20 days Weighted Moving Average
    dataset['WMA_5'] = dataset['Close'].rolling(window=5).apply(lambda x: np.dot(x, np.arange(1, 6))/15, raw=True)
    dataset['WMA_10'] = dataset['Close'].rolling(window=10).apply(lambda x: np.dot(x, np.arange(1, 11))/55, raw=True)
    dataset['WMA_20'] = dataset['Close'].rolling(window=20).apply(lambda x: np.dot(x, np.arange(1, 21))/210, raw=True)

    # Create MACD
    ema26 = dataset['Close'].ewm(span=26).mean()
    ema12 = dataset['Close'].ewm(span=12).mean()
    dataset['MACD'] = ema12 - ema26

    # RSI
    dataset['RSI'] = RSIIndicator(dataset['Close'].squeeze(), window=10).rsi()

    # Stochastic Oscillator (%K)
    stochastic_oscillator = StochasticOscillator(
        close=dataset['Close'],
        low=dataset['Low'],
        high=dataset['High'],
        window=14,
        smooth_window=3
    )
    dataset['Stochastic_K'] = stochastic_oscillator.stoch()  # %K
    dataset['Stochastic_D'] = dataset['Stochastic_K'].rolling(window=3).mean()  # %D (smoothed %K)

    # Williams %R
    dataset['Williams_R'] = (dataset['High'].rolling(window=14).max() - dataset['Close']) / \
                            (dataset['High'].rolling(window=14).max() - dataset['Low'].rolling(window=14).min())

    # High-Low Average and Close-Open
    dataset['Close_minus_Open'] = dataset['Close'] - dataset['Open']
    dataset['High_minus_Low'] = dataset['High'] - dataset['Low']

    # Commodity Channel Index (CCI)
    TP = (dataset['High'] + dataset['Low'] + dataset['Close']) / 3
    SMA = TP.rolling(window=10).mean()
    mad = TP.rolling(window=10).apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
    dataset['CCI'] = (TP - SMA) / (0.015 * mad)

    # Momentum
    dataset['Momentum'] = dataset['Close'].diff(periods=10)

    # Apply shifting if shift_values is True
    if shift_values:
        indicator_columns = [
            'SMA_5', 'SMA_10', 'SMA_20',
            'WMA_5', 'WMA_10', 'WMA_20',
            'MACD', 'RSI', 'Stochastic_K', 'Stochastic_D',
            'Williams_R', 'Close_minus_Open', 'High_minus_Low',
            'CCI', 'Momentum'
        ]
        dataset[indicator_columns] = dataset[indicator_columns].shift(1)

    return dataset

In [37]:
### ADD TECHNICAL INDICATORS
new_data = get_technical_indicators(data)

## ADD ECONOMIC INDICATORS
eco_data = pd.read_csv("../input/eco_data_daily_clean.csv")
# Rename date column
eco_data.rename(columns={"DATE": "Date"}, inplace=True)
# Convert date to datetime
eco_data["Date"] = pd.to_datetime(eco_data["Date"])


# Create final database
final_database = pd.merge(new_data, eco_data, on="Date", how="right")

# Drop Open", "High", "Low", "Volume", "Adj.Close","CAC40"
final_database.drop(["Open", "High", "Low", "Volume",  "CAC40"], axis=1, inplace=True)


In [38]:
final_database.head()

,Date,Close,Return,SMA_5,SMA_10,SMA_20,WMA_5,WMA_10,WMA_20,MACD,...,IRL,IRS,GDP,CPIH_YTYPCT,IRCB,UNR,YPH,UNR_us,IRCB_us,CPI_us
0,2000-01-03,5917.370117,-0.006896,5862.024023,5721.551025,5620.770020,5876.351986,5799.456934,5691.589455,66.063532,...,5.578081,3.558156,1.447856e+12,1.567372,2.516484,9.098174,4.838564e+11,4.047578,NaN,0.694063
1,2000-01-04,5672.020020,-0.042347,5874.998047,5763.048047,5643.236035,5894.800684,5835.060405,5719.837084,71.222855,...,5.576022,3.566075,1.448057e+12,1.566982,2.524725,9.093994,4.839066e+11,4.046468,NaN,0.694122
2,2000-01-05,5479.700195,-0.034495,5842.508008,5778.139063,5651.338525,5827.141341,5818.509854,5722.578416,57.911439,...,5.573963,3.573995,1.448258e+12,1.566592,2.532967,9.089814,4.839568e+11,4.045357,NaN,0.694182
3,2000-01-06,5450.109863,-0.005415,5773.032031,5773.185059,5648.598047,5706.205404,5764.248242,5706.231908,33.835439,...,5.571903,3.581915,1.448459e+12,1.566202,2.541209,9.085634,4.840070e+11,4.044247,NaN,0.694242
4,2000-01-07,5539.609863,0.016288,5695.504004,5756.067041,5646.747534,5598.564681,5705.507298,5687.328271,12.702531,...,5.569844,3.589835,1.448660e+12,1.565812,2.549451,9.081453,4.840572e+11,4.043137,NaN,0.694302


In [39]:
final_database.tail()

,Date,Close,Return,SMA_5,SMA_10,SMA_20,WMA_5,WMA_10,WMA_20,MACD,...,IRL,IRS,GDP,CPIH_YTYPCT,IRCB,UNR,YPH,UNR_us,IRCB_us,CPI_us
6343,2024-10-24,7503.279785,0.000773,7553.117969,7550.110010,7574.947949,7536.421322,7544.103640,7551.957068,0.571329,...,3.003752,2.992877,2.966591e+12,2.012639,2.937500,7.446300,1.010765e+12,4.161497,4.567500,1.292143
6344,2024-10-25,7497.540039,-0.000765,7537.027930,7546.279004,7563.007446,7519.808594,7535.589054,7545.131529,-2.963031,...,3.002665,2.990160,2.966774e+12,2.011326,2.934783,7.448313,1.010840e+12,4.161385,4.564783,1.292235
6345,2024-10-28,7556.939941,0.007891,7513.925977,7538.243994,7548.294946,7506.645964,7526.727424,7538.896538,-6.156221,...,2.999405,2.982008,2.967323e+12,2.007388,2.926630,7.454352,1.011064e+12,4.161049,4.556630,1.292511
6346,2024-10-29,7511.109863,-0.006083,7518.067969,7533.731982,7544.354443,7520.983952,7530.126687,7539.719871,-3.849398,...,2.998318,2.979290,2.967506e+12,2.006075,2.923913,7.456365,1.011138e+12,4.160936,4.553913,1.292603
6347,2024-10-30,7428.359863,-0.011078,7513.269922,7532.645947,7541.206445,7518.664583,7526.013574,7536.553720,-5.654151,...,2.997231,2.976573,2.967689e+12,2.004762,2.921196,7.458379,1.011213e+12,4.160824,4.551196,1.292695


In [40]:
final_database.columns

Index(['Date', 'Close', 'Return', 'SMA_5', 'SMA_10', 'SMA_20', 'WMA_5',
       'WMA_10', 'WMA_20', 'MACD', 'RSI', 'Stochastic_K', 'Stochastic_D',
       'Williams_R', 'Close_minus_Open', 'High_minus_Low', 'CCI', 'Momentum',
       'GDPV', 'IRL', 'IRS', 'GDP', 'CPIH_YTYPCT', 'IRCB', 'UNR', 'YPH',
       'UNR_us', 'IRCB_us', 'CPI_us'],
      dtype='object')

In [41]:
# Check for missing values
final_database.isnull().sum()

Date                  0
Close                 0
Return                0
SMA_5                 0
SMA_10                0
SMA_20                0
WMA_5                 0
WMA_10                0
WMA_20                0
MACD                  0
RSI                   0
Stochastic_K          0
Stochastic_D          0
Williams_R            0
Close_minus_Open      0
High_minus_Low        0
CCI                   0
Momentum              0
GDPV                  0
IRL                   0
IRS                   0
GDP                   0
CPIH_YTYPCT           0
IRCB                  0
UNR                   0
YPH                   0
UNR_us                0
IRCB_us             886
CPI_us                0
dtype: int64

In [42]:
# Save the data
final_database.to_csv("final_database.csv", index=False)